In [5]:
#importing dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy

In [11]:
# creat a set of random latitude and longitude combinations
lats = np.random.uniform(low=-90.000, high = 90.000, size = 1500)
lngs = np.random.uniform(low = -180.000, high = 180.000, size =1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [12]:
# add the latitudes and the longitudes to a list
coordinates = list(lat_lngs)

In [16]:
# use the tuple fucntion to display the latitude and longitude combiniations
cities = []
for coordinate in coordinates:
    city = (citipy.nearest_city(coordinate[0],coordinate[1]).city_name
    # if the city is unique then we will add it to the cities list
    if city not in cities:
            cities.appendy(city)

SyntaxError: invalid syntax (<ipython-input-16-367def8cb84a>, line 6)